In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.gaussian_process.kernels import ConstantKernel, DotProduct, ExpSineSquared, Matern, RBF, RationalQuadratic, WhiteKernel, Product, Sum
from tqdm import tqdm
import warnings

from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=RuntimeWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:
file_name = "data/selected_kernel/your_kernel_file.txt"

with open(file_name, "r") as file:
    file_string = file.read().strip()

kernel_optimized = eval(file_string)

print(kernel_optimized)

In [ ]:
time = 300

data = f"data/input/your_file.csv"
df = pd.read_csv(data)

In [ ]:
data_total = np.array(df.combined_data2.values)
data = data_total[:int(time)]
data = data.reshape(-1, 1)

CRdata = data/1000
CRdata  = np.ravel(CRdata)

In [ ]:
plt.figure(1, figsize=(11, 7), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data, linewidth = 3)
plt.legend(['Time Series'])
plt.grid(color='k', linestyle='--', linewidth=0.1)
plt.ylabel('Data', fontsize = 14)
plt.xlabel('Time Step', fontsize = 14)

In [ ]:
trainL = len(CRdata) - 60
t = np.linspace(1,len(CRdata),len(CRdata))
t = t.reshape(len(t),1)
t = np.atleast_2d(t)

t_tr  = t[0:trainL]
t_test = t[trainL:]

CR_tr = CRdata[0:trainL]
CR_test = CRdata[trainL:]

In [ ]:
def GPR_Model(CR_tr, t_tr, CR_test, t_test, kernel_optimized):
    t = np.append(t_tr, t_test).reshape(-1, 1)
    CRdata = np.append(CR_tr, CR_test)

    kernel = kernel_optimized

    model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20, alpha=1e-2, normalize_y=True)
    model.fit(t_tr, CR_tr)
    params = model.kernel_.get_params()

    R2_tr = model.score(t_tr, CR_tr)
    R2 = model.score(t, CRdata)
    R2_test = model.score(t_test, CR_test)

    CRpred_tr, sigma_tr = model.predict(t_tr, return_std=True)
    CRpred_test, sigma_test = model.predict(t_test, return_std=True)
    CRpred, sigma = model.predict(t, return_std=True)

    mse = np.mean(((CRpred_tr-CR_tr)*1000)**2)
    lml = model.log_marginal_likelihood_value_
    std = np.sqrt(mse)

    return {
        'CRpred_tr': CRpred_tr,
        'sigma_tr': sigma_tr,
        'CRpred_test': CRpred_test,
        'sigma_test': sigma_test,
        'CRpred': CRpred,
        'sigma': sigma,
        't': t,
        'CRdata': CRdata,
        'mse': mse,
        'std': std,
        'r2_tr': R2_tr,
        'r2_combined': R2,
        'r2_test': R2_test,
        'lml': lml,
        'kernel': model.kernel_
    }

In [ ]:
def find_best_model(CR_tr, t_tr, CR_test, t_test, kernel_optimized, patience=20):
    best_result = None
    no_improvement_count = 0

    with tqdm(total=patience, desc="Progress") as pbar:
        while no_improvement_count < patience:
            result = GPR_Model(CR_tr, t_tr, CR_test, t_test, kernel_optimized)

            if best_result is None or result['mse'] > best_result['mse']:
                best_result = result
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                pbar.update(1)

    return best_result

In [ ]:
best_result = find_best_model(CR_tr, t_tr, CR_test, t_test, kernel_optimized)

In [ ]:
print("Best learned kernel:", best_result['kernel'])
print("Best MSE:", best_result['mse'])
print("Best STD:", best_result['std'])
print("Best R² train:", best_result['r2_tr'])
print("Best R²:", best_result['r2_combined'])
print("Best R² test:", best_result['r2_test'])
print("Best lml: ", best_result['lml'])

In [ ]:
def plot_gpr(CRdata, CRpred, sigma, t, t_test):
  plt.figure(1, figsize=(20, 7), dpi=80, facecolor='w', edgecolor='k')
  mpl.style.use('default')

  plt.plot(t, CRdata*1000, 'k', markersize=3, mfc='none', linewidth=1, label='Time Series')
  plt.axvline(x=t_test[0], linestyle='--', color='black')
  plt.plot(t, CRpred*1000, 'b-', label='Prediction via GPR model', linewidth=3)
  plt.fill(np.concatenate([t, t[::-1]]),
          np.concatenate([CRpred*1000 - 1.9600 * sigma*1000,
                          (CRpred*1000 + 1.9600 * sigma*1000)[::-1]]),
          alpha=.25, fc='b', ec='None', label='Confidence interval - 95%')
  plt.legend(loc='upper left', fontsize=14)
  plt.grid(color='k', linestyle='--', linewidth=0.1)
  plt.ylabel('Data', fontsize=14)
  plt.xlabel('Time Step', fontsize=14)
  plt.gca().spines['top'].set_visible(False)
  plt.gca().spines['right'].set_visible(False)

  plt.show()

In [ ]:
plot_gpr(best_result['CRdata'], best_result['CRpred'], best_result['sigma'], best_result['t'], t_test)